In [1]:
import os
import pandas as pd
from pycytominer.cyto_utils.cells import SingleCells

In [2]:
# inputs
sql_file = "../data/SQ00014613.sqlite"
metadata_dir = "../data/metadata/"
barcode_platemap = "../data/barcode_platemap.csv"

In [ ]:
# Loading appropriate platemaps with given plate data
plate = os.path.basename(sql_file).rsplit(".", 1)
barcode_platemap_df = pd.read_csv(barcode_platemap)
platemap = barcode_platemap_df.query(
    "Assay_Plate_Barcode == @plate"
).Plate_Map_Name.values[0]
platemap_file = os.path.join(metadata_dir, "platemap", "{}.csv".format(platemap))
platemap_df = pd.read_csv(platemap_file)
platemap_df

In [ ]:
# TODO: generate a configuration file that contains the default parameters
# -- Configurations will be in the snakefile under the "param:" rule attribute
sqlite_file = "sqlite:///{}".format(sql_file)
strata = ["Image_Metadata_Plate", "Image_Metadata_Well"]
image_cols = ["TableNumber", "ImageNumber"]
single_cell_profile = SingleCells(
    sqlite_file, strata=strata, image_cols=image_cols, fields_of_view_feature="Image_Metadata_Well"
)

In [ ]:
print("Counting cells within each well")
cell_count_outfile = "cellcount.csv"
cell_count_df = single_cell_profile.count_cells()

print("Saving cell counts in: {}".format(cell_count_outfile))
cell_count_df = cell_count_df.merge(
    platemap_df, left_on="Image_Metadata_Well", right_on="well_position"
).drop(["WellRow", "WellCol", "well_position"], axis="columns")
cell_count_df.to_csv(cell_count_outfile, sep="\t", index=False) 

In [ ]:
aggregate_outfile = "./test.csv.gz"
ap = single_cell_profile.aggregate_profiles(output_file=aggregate_outfile, compression_options="gzip")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./results/preprocessing/SQ00014613.aggregate.csv.gz")
df2 = pd.read_csv("./data/barcode_platemap.csv")

In [ ]:
df.columns.values.tolist()

In [ ]:
df2.columns.tolist()

In [1]:
from pycytominer.annotate import annotate
from pycytominer.normalize import normalize

In [2]:
aggregated_profiles = "./results/preprocessing/SQ00014613.aggregate.csv.gz"
annotated_outfile = "./results/preprocessing/SQ00014613.annotated.csv.gz"
platemap_df = "./data/barcode_platemap.csv"

# this needs to be in configuration files
annotate(
    profiles=aggregated_profiles,
    platemap=platemap_df,
    # NOTE: Metadata_well_position was not in the dataframe
    join_on=["Metadata_Assay_Plate_Barcode", "Image_Metadata_Plate"],
    output_file=annotated_outfile,
    compression_options="gzip",
)

In [4]:
# Normalizing annotated profiles
annotated_file = "./results/preprocessing/SQ00014613.annotated.csv.gz"
normalized_out = "./SQ00014613.normalized.csv.gz"

# This needs to be in configuration files 
meta_features = [
    "Metadata_Plate",
    "Metadata_Well",
    'Metadata_Plate_Map_Name',
    'Metadata_Plate',
    'Metadata_Well',
    'Metadata_Object_Count',
]

normalize(
    profiles=annotated_file,
    features="infer",
    meta_features=meta_features,
    samples="all",
    method="mad_robustize",
    output_file=normalized_out,
    compression_options="gzip",
)
